The purpose of this code is to predict a sales for different items over the period of 90-days. It coincides with the kaggle competition found at https://www.kaggle.com/c/demand-forecasting-kernels-only. I chose to use a Neural net for this project. I was interested to see how it would perform compared to other projects that used ARIMA models.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import holidays 
from datetime import datetime, date
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/demand-forecasting-kernels-only/train.csv')
test = pd.read_csv('../input/demand-forecasting-kernels-only/test.csv')

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.info()

## Neural Net

In order to make a useful neural network, I extracted important information from the date column. I feature engineered columns that deal with day of the week, day in the year, whether a particular day was a holiday or not, and other features. Most features I transformed to be on a scaled from 1 to -1 by putting them through a cosine function. This gave my features a cyclical pattern that would repeat year over year. I felt like this would help my neural net take into account the cyclical nature of sales year to year and month to month.

In [ ]:
def createmodeldataframe(mydf):
    mydf["date"] = pd.to_datetime(mydf.date)
    mydf['dayofweek'] = mydf['date'].apply(lambda x:np.cos((2*np.pi*(x.weekday())/7)-.4))
    mydf['month'] = mydf['date'].apply(lambda x: np.cos((2*np.pi*x.month/12)-.1))
    mydf['dayinmonth'] = mydf['date'].apply(lambda x: np.cos((2*np.pi*x.day/30)-.1))
    mydf['dayinyear'] = mydf['date'].apply(lambda x: np.cos((2*np.pi*x.timetuple().tm_yday/365)-.01))
    mydf['year'] = mydf['date'].apply(lambda x: x.year)
    usholidays = holidays.US()
    mydf['US_holiday'] = mydf['date'].apply(lambda x: x in usholidays).astype(int)
    ohe = OneHotEncoder()
    df3 = pd.DataFrame(ohe.fit_transform(mydf[["store","item"]]).toarray())
    lastforecast=pd.concat([mydf,df3],axis=1)
    cleanforecast = lastforecast.drop(['date','store','item'], axis=1)
    return cleanforecast

In [ ]:
cleanforecast = createmodeldataframe(train)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
X_train = cleanforecast.drop('sales', axis=1)
y_train = cleanforecast['sales']

In [ ]:
#Split the data into training and testing sets.
#X_train, X_test, y_train, y_test = train_test_split(cleanforecast.drop('sales', axis=1), #
#                                                    cleanforecast.sales, test_size = 1,random_state = 101)


#Scaling the data because the data varies a lot(check X.describe())
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
#X_test = scaler.fit_transform(X_test)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [ ]:
model = Sequential()

model.add(Dense(66,activation='relu'))
model.add(Dense(40,activation='relu'))
model.add(Dropout(.3))
model.add(Dense(30,activation='relu'))
model.add(Dropout(.2))
model.add(Dense(20,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(15,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(x=X_train,y=y_train.values,
          batch_size=256,epochs=60)

In [ ]:
X_train.shape

## Submission

In [ ]:
#testdf = pd.read_csv("test.csv")
testdata = createmodeldataframe(test)
testids = testdata.id
testdata.drop('id',axis=1, inplace=True)
testdata = scaler.transform(testdata)
testpreds = model.predict(testdata)

In [ ]:
testpreds.shape

In [ ]:
news = testpreds.reshape((45000,))

In [ ]:
# Create submission
sub = pd.DataFrame({'id':testids,'sales':news})
sub.to_csv('submission.csv', index=False)